In [1]:
import pandas as pd
from paquo.projects import QuPathProject

from pathlib import Path
from paquo.projects import QuPathProject
from paquo.images import QuPathImageType
from paquo.classes import QuPathPathClass
from shapely.geometry import Point, Polygon, MultiPoint
import numpy as np
import matplotlib.pyplot as plt
from distutils.dir_util import copy_tree
import glob
import numpy as np
from collections import defaultdict


/tmp/ipykernel_27066/1052951956.py:11: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree


In [2]:
def get_all_projectQuPath_metadata(root_path):

    
    meta_list = glob.glob(root_path + '**/**/*Metadata_information.csv')
    project_by_image = {}
    main_df = pd.DataFrame()
    for path in meta_list:
        last_slash_pos = path.rfind('/')
        project_dir = path[:last_slash_pos + 1]
        qpproj_path = glob.glob(project_dir + '*.qpproj')[0] 
        df = pd.read_csv(path, index_col=0)
        df['qpproj'] = qpproj_path
        df_analyse = df[df.Analyze == True]
        #df_analyse = df
        main_df = pd.concat([main_df, df_analyse])
    main_df = main_df.reset_index(drop=True)
    return main_df

In [3]:
def get_qupath_project_by_image_name(dataset, image_name : str, root_path=None) ->str:

    project_name =  dataset[dataset['Image Name'].str.contains(image_name)]['qpproj'].iloc[0]
    return project_name
    '''
    print(f'DEBUG 1 project_name {project_name} ')
    project_name = 'ProjectQuPath_' + project_name[project_name.find('_')+1:]
    print(f'DEBUG 2 project_name {project_name} ')
    '''
    
    if root_path is not None:
        nissl_id = project_name[project_name.find('Nissl'):]
        project_name = root_path + nissl_id + '/' + project_name
        project_name = project_name
    return project_name

In [4]:
root_path ='/gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Analysis/'
#root_path = '/media/LNMC/LBR_Algorithm/Cellpose/QuPath_LayerBoundaries_GroundTruth_20231222'
#root_path ='/tmp/'
meta_dataset = get_all_projectQuPath_metadata(root_path)
len(meta_dataset)

279

In [5]:
#qupath_project_root_path = '/media/LNMC/LBR_Analysis/WithPredictions/'
qupath_project_root_path = root_path



pixel_size = 0.3460130331522824

layers_color = {"Layer 1": "#ff0000"
                , "Layer 2":"#0f00f0"
                , "Layer 3":"#0000ff"
                , "Layer 2/3":"#80801a"
                , "Layer 4":"#003333"
                , "Layer 5":"#00f5ff"
                , "Layer 6 a":"#663366"
                , "Layer 6 b":"#00ff00"
               }

In [6]:
with_prediction_list = glob.glob('/gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Production/20231102/datasets/Exported_Features/Dataframes/For_prediction/01122024/Predictions/*')
#with_prediction_list = glob.glob('/gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Production/20231102/datasets/Exported_Features/Dataframes/Ground_Truth/*.csv')
with_prediction_list
main_df_with_pred = pd.DataFrame()
for path in with_prediction_list:
    df = pd.read_csv(path)#, index_col=0)
    main_df_with_pred = pd.concat([main_df_with_pred, df])
main_df_with_pred = main_df_with_pred.reset_index(drop=False)    

def remove_space(a):
    return a.replace(' ','')

main_df_with_pred['Image'] = main_df_with_pred.apply(lambda row: remove_space(row['Image']), axis=1)

In [22]:
image_per_projects = defaultdict(list)
df1 = main_df_with_pred.drop_duplicates(subset=["Image"], keep="first")
for image_name in df1['Image']:
    image_name = image_name.replace(' ', '')
    qupath_project = get_qupath_project_by_image_name(meta_dataset, image_name, qupath_project_root_path)
    image_per_projects[qupath_project].append(image_name)



In [25]:
done = []
#nb_images = len(image_names)
#for image_name in image_names:
#    print(f' search {image_name} ')

#    qupath_project = get_qupath_project_by_image_name(dataset, image_name, qupath_project_root_path)

#    print(f'Image name {image_name}, qupath {qupath_project}')

for qupath_project in image_per_projects.keys():
    
    print(f'Open project {qupath_project}')
    with QuPathProject(qupath_project, mode='r+') as qp:
        print(f'process {image_per_projects[qupath_project]}')
        for image_name in image_per_projects[qupath_project]:
            image_ml_result = main_df_with_pred[main_df_with_pred['Image']==image_name]
            annotations = []
            for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 2/3",  "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
                layer_points = image_ml_result[image_ml_result['RF_prediction'] == layer][['Centroid X µm', 'Centroid Y µm']].to_numpy()
                layer_points = layer_points / pixel_size
                if layer_points.size > 0:
                    annotations.append({ layer: MultiPoint(layer_points)})
            assert len(annotations) > 0
            found = False
            for image in qp.images:
                image_name_with_space = image_name.replace('-', ' - ')
                if image.image_name.find(image_name_with_space) > -1:
                    for index, annotation in enumerate(annotations):
                        for name, roi in annotation.items():
                            path_class = QuPathPathClass(name = name + '_prediction', color=layers_color[name])

                            annotation = image.hierarchy.add_annotation(roi=roi, path_class=path_class)
                    found = True
                    done.append(image_name)
                if found:
                    break
        print(f'Close project')
    
    print(f'project closed')        

    print('INFO: Done')


    

Open project /gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Analysis/Nissl_5/ProjectQuPath_1443460_LH_Nissl_5/ProjectQuPath_1443460_LH_Nissl_5.qpproj
process ['SLD_0000612.vsi-20x_03', 'SLD_0000614.vsi-20x_02', 'SLD_0000616.vsi-20x_01', 'SLD_0000611.vsi-20x_02', 'SLD_0000612.vsi-20x_04', 'SLD_0000615.vsi-20x_02', 'SLD_0000613.vsi-20x_01', 'SLD_0000612.vsi-20x_01', 'SLD_0000614.vsi-20x_04', 'SLD_0000614.vsi-20x_01', 'SLD_0000613.vsi-20x_02', 'SLD_0000613.vsi-20x_04', 'SLD_0000611.vsi-20x_01', 'SLD_0000615.vsi-20x_01', 'SLD_0000610.vsi-20x_03', 'SLD_0000614.vsi-20x_03', 'SLD_0000615.vsi-20x_03']
Close project
14:55:07.862 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Analysis/Nissl_5/ProjectQuPath_1443460_LH_Nissl_5/ProjectQuPath_1443460_LH_Nissl_5.qpproj.tmp
14:55:07.895 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /gpfs/bbp.cscs.ch/project/proj53/LayerBoundari

In [20]:
done

['SLD_0000612.vsi-20x_03']

In [21]:
done = []
#nb_images = len(image_names)
#for image_name in image_names:
#    print(f' search {image_name} ')

#    qupath_project = get_qupath_project_by_image_name(dataset, image_name, qupath_project_root_path)

#    print(f'Image name {image_name}, qupath {qupath_project}')

for qupath_project in image_per_projects.keys():
    #if qupath_project.find('ProjectQuPath_01413829_RH_Nissl_2') == -1:
        
    #    continue
    print(f'Open project {qupath_project}')
    with QuPathProject(qupath_project, mode='r+') as qp:
        print(f'process {image_per_projects[qupath_project]}')
        for image_name in image_per_projects[qupath_project]:

            image_ml_result = image_ml_result[image_ml_result['Image']==image_name]

            annotations = []
            for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 2/3",  "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
            #for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
                layer_points = image_ml_result[image_ml_result['RF_prediction'] == layer][['Centroid X µm', 'Centroid Y µm']].to_numpy()
                #print(f'Layer points {layer_points}')
                layer_points = layer_points / pixel_size
                if layer_points.size > 0:
                    annotations.append({ layer: MultiPoint(layer_points)})

            assert len(annotations) > 0
            #print("readed", qp.name)  
            #print(f'---INFO: there are {len(qp.images)}  images in this project')
            found = False
            for image in qp.images:
                image_name_wih_space = image_name.replace('-', ' - ')
                if image.image_name.find(image_name_wih_space) > -1:
                    for index, annotation in enumerate(annotations):
                        for name, roi in annotation.items():
                            path_class = QuPathPathClass(name = name + '_prediction', color=layers_color[name])

                            annotation = image.hierarchy.add_annotation(roi=roi, path_class=path_class)
                    found = True
                    done.append(image_name)
                if found: 
                    break
        print(f'Close project')
    
    print(f'project closed')        

    print('INFO: Done')


    

Open project /gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Analysis/Nissl_5/ProjectQuPath_1443460_LH_Nissl_5/ProjectQuPath_1443460_LH_Nissl_5.qpproj
process ['SLD_0000612.vsi-20x_03']


Saving skipped for 'SLD_0000596.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_5/SLD_0000596.vsi' not reachable
Saving skipped for 'SLD_0000596.vsi - 20x_02': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_5/SLD_0000596.vsi' not reachable
Saving skipped for 'SLD_0000596.vsi - 20x_03': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_5/SLD_0000596.vsi' not reachable
Saving skipped for 'SLD_0000596.vsi - 20x_04': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_5/SLD_0000596.vsi' not reachable
Saving skipped for 'SLD_0000596.vsi - 20x_05': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_5/SLD_0000596.vsi' not reachable
Saving skipped for 'SLD_0000596.vsi - 20x_06': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_5/SLD_0000596.vsi' not reachable
Saving skipped for 'SLD_0000596.vsi - 20x_07': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_5/SLD_0000596.vsi'

Close project
13:40:32.660 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Analysis/Nissl_5/ProjectQuPath_1443460_LH_Nissl_5/ProjectQuPath_1443460_LH_Nissl_5.qpproj.tmp
13:40:32.704 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Analysis/Nissl_5/ProjectQuPath_1443460_LH_Nissl_5/ProjectQuPath_1443460_LH_Nissl_5.qpproj.backup
13:40:32.723 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Analysis/Nissl_5/ProjectQuPath_1443460_LH_Nissl_5/ProjectQuPath_1443460_LH_Nissl_5.qpproj
project closed
INFO: Done


In [81]:
done

['SLD_0000612.vsi-20x_03']

In [111]:
df = pd.read_csv('/media/LNMC/LBR_Algorithm/Cellpose/QuPath_LayerBoundaries_GroundTruth_20231222/QuPath_LayerBoundaries_GroundTruth_20231222_Metadata information.csv', index_col=0)

In [121]:
df=pd.read_csv('/gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Production/20231102/datasets/Exported_Features/Dataframes/Ground_Truth/Features_SLD_0000612.vsi-20x_01.csv')

In [123]:
df.isnull().values.any()

False